# 実験管理サンプルコード
* このnotebookの目的
    * 技術書典2019秋における実験管理を行うコード例

In [ ]:
%matplotlib inline

In [ ]:
import pathlib
import subprocess
import time
import datetime
import json

import numpy as np
import pandas as pd

import mxnet as mx
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn
from mxnet.gluon.contrib.estimator import estimator

from utils import manager, train_module, preprocessing
from utils.manager import Run, Experiment

import warnings

%load_ext autoreload
%autoreload 2

# 前処理用

In [ ]:
exp = manager.Experiment('Your_Data_Path/', 'normalize')

In [ ]:
setting = dict(
    resize=64
)

In [ ]:
preprocessing.data_preprocessing(exp, setting, tags=['resize'])

# 学習用

In [ ]:
exp = manager.Experiment('experiments', 'fashion_mlp')

In [ ]:
setting = dict(
    data_path = 'Your_Data_Path/normalize/0_resize/normalized_data.pkl',
    epochs = 10,
    gpu_count = 1,
    batch_size = 256,
    opt_params=dict(learning_rate = 0.01),    
    opt = 'sgd',
    model_params=dict(
        layers=[100, 20],
        bn=True,
        act='relu'
    ),
)

In [ ]:
train_module.train(exp, setting, tags=['mlp'])

In [ ]:
run = Run(exp, 1)

In [ ]:
with run.path.joinpath('setting.json').open(mode='r') as f:
    setting_0 = json.load(f)

In [ ]:
net = train_module.mlp(**setting_0['model_params'])

In [ ]:
net.load_params(str(run.path.joinpath('model-best.params')))

In [ ]:
commit_hash = subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD'])

In [ ]:
commit_hash.decode('utf-8')

In [ ]:
run = Run(exp, 8)

In [ ]:
run.delete()

In [ ]:
summary_df = exp.summary('log.pkl')

In [ ]:
train_acc_columns = [s for s in summary_df.columns if s.split('_')[-1] == 'train accuracy']

In [ ]:
summary_df[train_acc_columns].plot()